# INTRODUCCIÓN

#### En este ejercicio se va a trabajar la analitica exploratoria (EDA) sobre un dataset de variables para la estimacion de fraude en operaciones bancarias.
#### Para ello, vamos a empezar analizando el dataset disponible, el tipo de variables que lo componen y prepararlo para su posterior análisis y modelos de clasificación

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
%matplotlib inline 

RANDOM_SEED = 42

plt.style.use('bmh')

# EDA

#### Iniciamos el proceso de análisis, estudiando los datos disponibles

In [ ]:
df = pd.read_csv('./FraudInputDatav2.csv', encoding = "utf-16") #leemos el fichero de datos, codificado en utf-16
df.head()

In [ ]:
df.info()

#### Como podeis ver, se trata de un dataframe compuesto por 3 tipos de columnas, tenemos datos en coma flotante (float64), datos enteros (int64) y los datos tipo object son strings, es decir variables categóricas, las cuales posteriormente tendremos que linearizar

In [ ]:
#vamos a linealizar las variables categoricas usando un one-hot-enconding
#para eso el primer paso es encontrar todas las variables categoricas, seran aquellas que tengan el tipo object
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()
#como podeis ver el dataset tiene 8 variables categoricas. No linealizaremos FraudStr y sample ya que serán borradas posteriormente

In [ ]:
##realizamos la codificación
df_encoding = pd.get_dummies(obj_df, columns=['pm','Channel','Product','sl','le','Routing'])
df_encoding.head()
#como podeis ver se ha creado una columna por cada valor de las variables categoricas.
#Por ejemplo la variable payment ha sido convertida a 4 variables numéricas

In [ ]:
#obtenemos el resto de variables para concatenarlo en el df de trabajo
num_df = df.select_dtypes(include=['int64','float64']).copy()
df_num = pd.concat([df_encoding,num_df], axis=1).drop('ID',1).drop('random', 1).drop('sample', 1).drop('FraudStr', 1) #aprovecho para quitar columnas no necesarias


#### Vamos a estudiar la variable Value de forma especial, ya que es una variable que habitualmente puede arrojar mucha informacion respecto al fraude de la operación. Aunque la variable está anonimizada como vereis es una muy buena medida discriminatoria.
#### Podeis realizar estas mismas graficas sobre otros valores que creais de interés

In [ ]:
bins=160
plt.figure(figsize=(20,4))
plt.hist(df_num.Value[df_num.Fraud==1],bins=bins,normed=True,alpha=0.8,label='Fraud',color='red')
plt.hist(df_num.Value[df_num.Fraud==0],bins=bins,normed=True,alpha=0.8,label='Not Fraud',color='lightblue')
plt.legend(loc='upper right')
plt.xlabel('Valor')
plt.ylabel('% de Registros')
plt.title('Transacciones vs Valor')
plt.show()

In [ ]:
### Como puede verse esta variable puede llegar a ser muy discriminante. Vamos a estudiar el resto de variables

In [ ]:
# Estudiamos el resto de variables
y=df_num.Fraud
x=df_num.drop(['Fraud','Value'],axis=1) ## Quitamos la variable Value y la etiqueta Fraud

#### El primer punto es analizar la distribución de variables, tal como hemos visto en el apartado teórico. Lo que estamos buscando son variables cuya distribución difiera entre los casos de Fraude y NoFraude. Es decir, vamos a eliminar del dataset todas las variables cuya distribución sea muy similar entre ambos casos.
#### Para facilitar la visualización de resultados, debido al alto número de variables, crearemos las gráficas en dos grupos

In [ ]:
#Primer grupo de variables (0-60) La generación de estas gráficas lleva algo de tiempo de computo
x_scaled=(x-x.min())/(x.max()-x.min()) 
sub_df1=pd.concat([y,x_scaled.iloc[:,0:10]],axis=1)
sub_df2=pd.concat([y,x_scaled.iloc[:,10:20]],axis=1)
sub_df3=pd.concat([y,x_scaled.iloc[:,20:30]],axis=1)
sub_df4=pd.concat([y,x_scaled.iloc[:,30:40]],axis=1)
sub_df5=pd.concat([y,x_scaled.iloc[:,40:50]],axis=1)
sub_df6=pd.concat([y,x_scaled.iloc[:,50:60]],axis=1)
sub_df11=pd.melt(sub_df1,id_vars="Fraud",var_name="Variable",value_name='Valor')
sub_df22=pd.melt(sub_df2,id_vars="Fraud",var_name="Variable",value_name='Valor')
sub_df33=pd.melt(sub_df3,id_vars="Fraud",var_name="Variable",value_name='Valor')
sub_df44=pd.melt(sub_df4,id_vars="Fraud",var_name="Variable",value_name='Valor')
sub_df55=pd.melt(sub_df5,id_vars="Fraud",var_name="Variable",value_name='Valor')
sub_df66=pd.melt(sub_df6,id_vars="Fraud",var_name="Variable",value_name='Valor')
plt.figure(figsize=(20,8))
sns.violinplot(x="Variable",y="Valor",hue="Fraud",data=sub_df11, split=True)
plt.figure(figsize=(20,8))
sns.violinplot(x="Variable",y="Valor",hue="Fraud",data=sub_df22, split=True)
plt.figure(figsize=(20,8))
sns.violinplot(x="Variable",y="Valor",hue="Fraud",data=sub_df33, split=True)
plt.figure(figsize=(20,8))
sns.violinplot(x="Variable",y="Valor",hue="Fraud",data=sub_df44, split=True)
plt.figure(figsize=(20,8))
sns.violinplot(x="Variable",y="Valor",hue="Fraud",data=sub_df55, split=True)
plt.figure(figsize=(20,8))
sns.violinplot(x="Variable",y="Valor",hue="Fraud",data=sub_df66, split=True)

In [ ]:
#Segundo grupo de variables (60--) La generación de estas gráficas lleva algo de tiempo de computo
x_scaled=(x-x.min())/(x.max()-x.min()) 
i=60
sub_df1=pd.concat([y,x_scaled.iloc[:,i+0:i+10]],axis=1)
sub_df2=pd.concat([y,x_scaled.iloc[:,i+10:i+20]],axis=1)
sub_df3=pd.concat([y,x_scaled.iloc[:,i+20:i+30]],axis=1)
sub_df4=pd.concat([y,x_scaled.iloc[:,i+30:i+40]],axis=1)
sub_df5=pd.concat([y,x_scaled.iloc[:,i+40:i+50]],axis=1)

sub_df11=pd.melt(sub_df1,id_vars="Fraud",var_name="Variable",value_name='Valor')
sub_df22=pd.melt(sub_df2,id_vars="Fraud",var_name="Variable",value_name='Valor')
sub_df33=pd.melt(sub_df3,id_vars="Fraud",var_name="Variable",value_name='Valor')
sub_df44=pd.melt(sub_df4,id_vars="Fraud",var_name="Variable",value_name='Valor')
sub_df55=pd.melt(sub_df5,id_vars="Fraud",var_name="Variable",value_name='Valor')

plt.figure(figsize=(20,8))
sns.violinplot(x="Variable",y="Valor",hue="Fraud",data=sub_df11, split=True)
plt.figure(figsize=(20,8))
sns.violinplot(x="Variable",y="Valor",hue="Fraud",data=sub_df22, split=True)
plt.figure(figsize=(20,8))
sns.violinplot(x="Variable",y="Valor",hue="Fraud",data=sub_df33, split=True)
plt.figure(figsize=(20,8))
sns.violinplot(x="Variable",y="Valor",hue="Fraud",data=sub_df44, split=True)
plt.figure(figsize=(20,8))
sns.violinplot(x="Variable",y="Valor",hue="Fraud",data=sub_df55, split=True)


In [ ]:
#borramos columnas con una distribución uniforme entre fraude y no fraude
df_features=df_num.drop(['Channel_BA','Channel_BB','Channel_BC','Channel_CA','Channel_D','Channel_E','Product_A','Product_B','Product_CA','Product_D','Product_E','Product_G','Product_Z','sl_JP','sl_NZ','Routing_0','Routing_aoldialup','Routing_aolpop','Routing_aolproxy','Routing_cache proxy','Routing_pop','Routing_regional proxy','Routing_satellite','ac','ab','N1','NCE','QualifiedGood','Risk_Email','Risk_CEP','Risk_Card','Risk_FistName','Risk_LastName','Risk_IP_Class','Risk_IP_Domain','Risk_IP_Carrier'],axis=1)
df_features = pd.concat([df_features,df_num[['Value']]], axis=1) #incluimos el campo Value que fue extraido anteriormente

# BALANCEO DE DATASET Y NORMALIZACIÓN DE DATOS

#### En este punto hemos dejado el dataset con solamente aquellas variables que a-priori pueden aportar algún tipo de información respecto al fraude. Ahora vamos a analizar el dataset a nivel de tipo de información.
#### Como vereis a continuación, un problema común que nos encontramos en la analítica operacional es que el dataset está desbalanceado, es decir existen muchas más muestras de una clase u etiqueta que de otra.
#### Esto es especialmente importante cuando se detectan anomalías o situaciones como la de este ejercicio de detección de fraude. Existen muchas más transacciones legíticas que fraudulentas

In [ ]:
count_classes = pd.value_counts(df_features['Fraud'], sort = True).sort_index()
labels = 'Fraude', 'No Fraude'
sizes = [count_classes[1]/(count_classes[1]+count_classes[0]), count_classes[0]/(count_classes[1]+count_classes[0])]
explode = (0, 0.5,)  
colors = ['red', 'blue']
fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, colors=colors, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=45)
ax1.axis('equal')  
plt.title("Distribución del Dataset en clases etiquetadas")
plt.show()

In [ ]:
df_features.shape

#### Como podeis ver, el dataset está muy desbalanceado. Existen técnicas para mejorar el balanceo del mismo, posteriormente veremos algunas, pero lo más importante es que este tipo de situaciones deben ser tenidas en cuenta a la hora de analizar los resultados de los modelos que apliquemos. Lo veremos cuando analicemos los resultados obtenidos.

#### El siguiente paso va a saer la normalización del dataset

In [ ]:
#Vamos a analizar dentro del dataset aquellas columnas cuyo valor minimo sea menor que -1 y valor máximo mayor que 1.
#Para ello vamos a apoyarnos en la funcion describe de un DataFrame que nos proporciona toda esta información y mas
tt = df_features.describe().transpose()
tt[(tt['max']>1) & (tt['min']< -1)]

In [ ]:
#Como podeis ver existen 3 variables que es necesario normalizar, el resto son correctas
columns_to_norm = ['Risk_Item','Risk_ProductType','Risk_Provider']

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler() # se encarga de normalizar una variable entre -1 y 1
df_features[columns_to_norm]=min_max_scaler.fit_transform(df_features[columns_to_norm])


In [ ]:
tt = df_features.describe().transpose()
tt[(tt['max']>1) & (tt['min']< -1)]
#Ya no quedan variables que normalizar, todas están en los rangos esperados

In [ ]:
#Generamos una función de ayuda para el resto de módulos de cara a visualizar las matrices de confusión
from sklearn.metrics import confusion_matrix, classification_report, auc, precision_recall_curve, roc_curve
def plot_confusion_matrix(y_test, pred):
    
    y_test_legit = y_test.value_counts()[0]
    y_test_fraud = y_test.value_counts()[1]
    
    cfn_matrix = confusion_matrix(y_test, pred)
    cfn_norm_matrix = np.array([[1.0 / y_test_legit,1.0/y_test_legit],[1.0/y_test_fraud,1.0/y_test_fraud]])
    norm_cfn_matrix = cfn_matrix * cfn_norm_matrix

    fig = plt.figure(figsize=(12,5))
    ax = fig.add_subplot(1,2,1)
    sns.heatmap(cfn_matrix,cmap='coolwarm_r',linewidths=0.5,annot=True,ax=ax)
    plt.title('Matriz de Confusión')
    plt.ylabel('Categorias reales')
    plt.xlabel('Categorias estimadas')

    ax = fig.add_subplot(1,2,2)
    sns.heatmap(norm_cfn_matrix,cmap='coolwarm_r',linewidths=0.5,annot=True,ax=ax)

    plt.title('Matriz de Confusión normalizada')
    plt.ylabel('Categorias reales')
    plt.xlabel('Categorias estimadas')
    plt.show()
    
    print('---Report de clasificación---')
    print(classification_report(y_test,pred))

# PREPARACIÓN DE DATOS PARA APRENDIZAJE Y LANZAMIENTO DE MODELO

#### El primer paso es separar los datos en datos de entrenamiento y de test. En este ejercicio se usa una aproximación sencilla, separando los datos en 80% training, 20% test. Se pueden aplicar métodos más avanzados, incluyendo datos de validación posteriores o realizar una randominzación mayor de los datos a obtener, pero para este caso es suficiente esta aproximación.
#### Asi mismo, es necesario separar los datos en variables y etiquetas (X_train, X_test, Y_train, Y_test)

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test = train_test_split(df_features, test_size=0.2, random_state=RANDOM_SEED)
Y_train = X_train['Fraud']
X_train = X_train.drop(['Fraud'], axis=1)
Y_test = X_test['Fraud']
X_test = X_test.drop(['Fraud'], axis=1)

#### Vamos aplicar de inicio un modelo de regresión. Se trata de un modelo de regularización lineal que utiliza un Stochastic Gradient Descent (SGD). Es un modelo suficientemente robusto para una primera prueba aunque a futuro probaremos modelos más complejos.
#### Para más información de este modelo consultar 
##### http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

In [ ]:
from sklearn import metrics

sgd_clf=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

sgd_clf.fit(X_train, Y_train) 
Y_train_predicted=sgd_clf.predict(X_train)
Y_test_predicted=sgd_clf.predict(X_test)

plot_confusion_matrix(Y_test, Y_test_predicted)

##### A simple vista los resultados del modelo son bastante buenos. En el resultado medio vemos una precisión del 98% y un recall del 98% también. Recordemos que el término precisión nos indica la capacidad del clasificador para no etiquetar como positivo una muestra negativa. Es decir, la capacidad del modelo para no dar registros correctos como fraudulentos. El término recall nos indica la capacidad del clasificador para encontrar todas las muestra positivas.
##### Pero si vemos en más detalle los resultados, vemos cómo en el caso de la clase 1, es decir Fraude, los resultados no son tan buenos. Si combinamos estos datos con la matriz de confusión (no normalizada) vemos cómo el modelo estima correctamente 630 registros sobre un total de aproximadamente 720, pero que etiqueta como fraudulentas 150 operaciones que en realidad son correctas.

#### Dependiendo de cómo se operativice el modelo, esto puede ser un buen o mal resultado. Como pregunta abierta, ¿es mejor reducir el rendimiento en los registros correctos a  cambio de mejorar el rendimiento del modelo en operaciones fraudulentas? Veamos algunos ejemplos

# DESBALANCEAR EL DATASET

#### Una estrategia a seguir es reducir el número de muestras correctas, para intentar balancear el dataset. Al no ser un dataset especialmente grande es necesario tener cuidado con esta operación, ya que reducir drasticamente el número de muestras penalizará el modelo al no tener suficientes datos para su aprendizaje.

In [ ]:
from sklearn.utils import shuffle

Train_Data= pd.concat([X_train, Y_train], axis=1)
X_1 =Train_Data[ Train_Data["Fraud"]==1 ]
X_0=Train_Data[Train_Data["Fraud"]==0]

X_0=shuffle(X_0,random_state=42).reset_index(drop=True)
X_1=shuffle(X_1,random_state=42).reset_index(drop=True)

ALPHA=1 #sobre esta variable podeis jugar para cambiar la distribución del dataset

X_0=X_0.iloc[:round(len(X_1)*ALPHA),:]
data_d=pd.concat([X_1, X_0])

count_classes = pd.value_counts(data_d['Fraud'], sort = True).sort_index()
labels = 'Fraude', 'No Fraude'
sizes = [count_classes[1]/(count_classes[1]+count_classes[0]), count_classes[0]/(count_classes[1]+count_classes[0])]
explode = (0, 0.05,)
colors = ['red', 'blue']
fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, colors=colors, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=45)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title("Distribución del dataset en clases")
plt.show()

In [ ]:
data_d.shape

In [ ]:
Y_d=data_d['Fraud']
X_d=data_d.drop(['Fraud'],axis=1)

In [ ]:
sgd_clf_d=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

sgd_clf_d.fit(X_d, Y_d) 


Y_test_predicted=sgd_clf_d.predict(X_test)

plot_confusion_matrix(Y_test, Y_test_predicted)

##### Si observamos este escenario, donde hemos reducido el número de muestas de entrenamiento a un 50% correctas, 50% fraudulentas, por un lado el número de falsos negativos en la clase fraude ha disminuido de 95 a 20, mientras que el número de falsos positivos, entendidos como el número de registros correctos que han sido detectados como fraudulentos ha aumentado de 150 a 950 aproximadamente. 

##### Para finalizar, vamos a ejecutar el mismo dataset balanceado con un modelo algo más complejo y potente como es el RandomForest, y analicemos los resultados

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
rf =RandomForestClassifier(n_estimators=100, max_depth=None, random_state=0, n_jobs=-1)
rf.fit(X_d, Y_d) 
Y_test_predicted=rf.predict(X_test)

plot_confusion_matrix(Y_test, Y_test_predicted)

#### En términos generales se trata de un modelo que arroja mejores resultados, aunque si miramos la matriz de confusión ha aumentado de 20 a 38 los falsos negativos, y ha disminuido de 950 a 350 aproximadamente en los falsos positivos